In [1]:
!pip install requests

Defaulting to user installation because normal site-packages is not writeable


In [2]:

import requests
import datetime

In [3]:
import base64

In [4]:
client_id = '3159028618334df686b6d604c56fc665'
client_secret = 'ffd7dc7920ac41e8a83c84f0f0a241a6'

In [5]:
# do a lookup for a token
# this token is for future requests (token expires at some point)

In [6]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret

    def get_client_credentials(self):
        """
        Returns a base64 encoded string
        """
        client_id = self.client_id
        client_secret = self.client_secret
        if client_secret == None or client_id == None:
            raise Exception("You must set client_id and client_secret")
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return {
            "Authorization": f"Basic {client_creds_b64}"
        }
    
    def get_token_data(self):
        return {
            "grant_type": "client_credentials"
        } 
    
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        r = requests.post(token_url, data=token_data, headers=token_headers)
        if r.status_code not in range(200, 299):
            return False
        data = r.json()
        now = datetime.datetime.now()
        access_token = data['access_token']
        expires_in = data['expires_in'] # seconds
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True

In [7]:
spotify = SpotifyAPI(client_id, client_secret)

In [8]:

spotify.perform_auth()

True

In [9]:

spotify.access_token

'BQB1QRsLHtcWFiPK4AF6srTpapvbkKmd1k3ZK7dviJQ8tiuaDYxwLaOzRVhLdKo9wd9c3pAu_tkcwFJjNxM'

In [10]:
# spotify.search

In [14]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

cid ="3159028618334df686b6d604c56fc665" 
secret = "ffd7dc7920ac41e8a83c84f0f0a241a6"


client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [22]:
import timeit
start = timeit.default_timer()

# create empty lists where the results are going to be stored
artist_name = []
track_name = []
popularity = []
track_id = []

for i in range(0,1000,50):
    track_results = sp.search(q='year:2019', type='track', limit=50,offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        popularity.append(t['popularity'])
      

stop = timeit.default_timer()
print ('Time to run this code (in seconds):', stop - start)

Time to run this code (in seconds): 3.01299929999999


In [23]:
print('number of elements in the track_id list:', len(track_id))

number of elements in the track_id list: 1000


In [24]:
import pandas as pd

df_tracks = pd.DataFrame({'artist_name':artist_name,'track_name':track_name,'track_id':track_id,'popularity':popularity})
print(df_tracks.shape)
df_tracks.head()

(1000, 4)


,artist_name,track_name,track_id,popularity
0,Harry Styles,Watermelon Sugar,6UelLqGlWMcVH1E5c4H7lY,97
1,Roddy Ricch,The Box,0nbXyq5TXYPCO7pr3N8S4I,92
2,Surfaces,Sunday Best,1Cv1YLb4q0RzL6pybtaMLo,91
3,Post Malone,Circles,21jGcNKet2qwijlDFuPiPb,91
4,Juice WRLD,Robbery,6Hj9jySrnFppAI0sEMCZpJ,88


In [25]:
df_tracks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   artist_name  1000 non-null   object
 1   track_name   1000 non-null   object
 2   track_id     1000 non-null   object
 3   popularity   1000 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 31.4+ KB


In [26]:
# group the entries by artist_name and track_name and check for duplicates

grouped = df_tracks.groupby(['artist_name','track_name'], as_index=True).size()
grouped[grouped > 1].count()

29

In [27]:
df_tracks.drop_duplicates(subset=['artist_name','track_name'], inplace=True)
# doing the same grouping as before to verify the solution
grouped_after_dropping = df_tracks.groupby(['artist_name','track_name'], as_index=True).size()
grouped_after_dropping[grouped_after_dropping > 1].count()


0

In [28]:
df_tracks.shape


(971, 4)

In [29]:
# Audio Features

# again measuring the time
start = timeit.default_timer()

# empty list, batchsize and the counter for None results
rows = []
batchsize = 100
None_counter = 0

for i in range(0,len(df_tracks['track_id']),batchsize):
    batch = df_tracks['track_id'][i:i+batchsize]
    feature_results = sp.audio_features(batch)
    for i, t in enumerate(feature_results):
        if t == None:
            None_counter = None_counter + 1
        else:
            rows.append(t)
            
print('Number of tracks where no audio features were available:',None_counter)

stop = timeit.default_timer()
print ('Time to run this code (in seconds):',stop - start)


Number of tracks where no audio features were available: 0
Time to run this code (in seconds): 1.2871819000000073


In [30]:
print('number of elements in the track_id list:', len(rows))

number of elements in the track_id list: 971


In [31]:
df_audio_features = pd.DataFrame.from_dict(rows,orient='columns')
print("Shape of the dataset:", df_audio_features.shape)
df_audio_features.head()

Shape of the dataset: (971, 18)


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.548,0.816,0,-4.209,1,0.0465,0.122,0.00000,0.3350,0.557,95.390,audio_features,6UelLqGlWMcVH1E5c4H7lY,spotify:track:6UelLqGlWMcVH1E5c4H7lY,https://api.spotify.com/v1/tracks/6UelLqGlWMcV...,https://api.spotify.com/v1/audio-analysis/6Uel...,174000,4
1,0.896,0.586,10,-6.687,0,0.0559,0.104,0.00000,0.7900,0.642,116.971,audio_features,0nbXyq5TXYPCO7pr3N8S4I,spotify:track:0nbXyq5TXYPCO7pr3N8S4I,https://api.spotify.com/v1/tracks/0nbXyq5TXYPC...,https://api.spotify.com/v1/audio-analysis/0nbX...,196653,4
2,0.878,0.525,5,-6.832,1,0.0578,0.183,0.00000,0.0714,0.694,112.022,audio_features,1Cv1YLb4q0RzL6pybtaMLo,spotify:track:1Cv1YLb4q0RzL6pybtaMLo,https://api.spotify.com/v1/tracks/1Cv1YLb4q0Rz...,https://api.spotify.com/v1/audio-analysis/1Cv1...,158571,4
3,0.695,0.762,0,-3.497,1,0.0395,0.192,0.00244,0.0863,0.553,120.042,audio_features,21jGcNKet2qwijlDFuPiPb,spotify:track:21jGcNKet2qwijlDFuPiPb,https://api.spotify.com/v1/tracks/21jGcNKet2qw...,https://api.spotify.com/v1/audio-analysis/21jG...,215280,4
4,0.685,0.692,2,-5.122,1,0.0457,0.328,0.00000,0.1530,0.578,159.966,audio_features,6Hj9jySrnFppAI0sEMCZpJ,spotify:track:6Hj9jySrnFppAI0sEMCZpJ,https://api.spotify.com/v1/tracks/6Hj9jySrnFpp...,https://api.spotify.com/v1/audio-analysis/6Hj9...,240051,5


In [32]:
df_audio_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 971 entries, 0 to 970
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   danceability      971 non-null    float64
 1   energy            971 non-null    float64
 2   key               971 non-null    int64  
 3   loudness          971 non-null    float64
 4   mode              971 non-null    int64  
 5   speechiness       971 non-null    float64
 6   acousticness      971 non-null    float64
 7   instrumentalness  971 non-null    float64
 8   liveness          971 non-null    float64
 9   valence           971 non-null    float64
 10  tempo             971 non-null    float64
 11  type              971 non-null    object 
 12  id                971 non-null    object 
 13  uri               971 non-null    object 
 14  track_href        971 non-null    object 
 15  analysis_url      971 non-null    object 
 16  duration_ms       971 non-null    int64  
 1

In [33]:
columns_to_drop = ['analysis_url','track_href','type','uri']
df_audio_features.drop(columns_to_drop, axis=1,inplace=True)

df_audio_features.rename(columns={'id': 'track_id'}, inplace=True)

df_audio_features.shape

(971, 14)

In [34]:
# merge both dataframes
# the 'inner' method will make sure that we only keep track IDs present in both datasets
df = pd.merge(df_tracks,df_audio_features,on='track_id',how='inner')
print("Shape of the dataset:", df_audio_features.shape)
df.head()

Shape of the dataset: (971, 14)


,artist_name,track_name,track_id,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Harry Styles,Watermelon Sugar,6UelLqGlWMcVH1E5c4H7lY,97,0.548,0.816,0,-4.209,1,0.0465,0.122,0.00000,0.3350,0.557,95.390,174000,4
1,Roddy Ricch,The Box,0nbXyq5TXYPCO7pr3N8S4I,92,0.896,0.586,10,-6.687,0,0.0559,0.104,0.00000,0.7900,0.642,116.971,196653,4
2,Surfaces,Sunday Best,1Cv1YLb4q0RzL6pybtaMLo,91,0.878,0.525,5,-6.832,1,0.0578,0.183,0.00000,0.0714,0.694,112.022,158571,4
3,Post Malone,Circles,21jGcNKet2qwijlDFuPiPb,91,0.695,0.762,0,-3.497,1,0.0395,0.192,0.00244,0.0863,0.553,120.042,215280,4
4,Juice WRLD,Robbery,6Hj9jySrnFppAI0sEMCZpJ,88,0.685,0.692,2,-5.122,1,0.0457,0.328,0.00000,0.1530,0.578,159.966,240051,5


In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 971 entries, 0 to 970
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   artist_name       971 non-null    object 
 1   track_name        971 non-null    object 
 2   track_id          971 non-null    object 
 3   popularity        971 non-null    int64  
 4   danceability      971 non-null    float64
 5   energy            971 non-null    float64
 6   key               971 non-null    int64  
 7   loudness          971 non-null    float64
 8   mode              971 non-null    int64  
 9   speechiness       971 non-null    float64
 10  acousticness      971 non-null    float64
 11  instrumentalness  971 non-null    float64
 12  liveness          971 non-null    float64
 13  valence           971 non-null    float64
 14  tempo             971 non-null    float64
 15  duration_ms       971 non-null    int64  
 16  time_signature    971 non-null    int64  
dt

In [36]:
df[df.duplicated(subset=['artist_name','track_name'],keep=False)]

,artist_name,track_name,track_id,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature


In [37]:
df.to_csv('SpotifyAudioFeatures08082018.csv')